In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime #for working with times objects
from datetime import timedelta #for working with times objects
import math
import random
import glob
from sklearn.model_selection import train_test_split

# Prepare dataset for new features

In [5]:
idle_time = pd.read_csv("Phil_with_idle_time_bike.csv")

C:\Users\Jonas\AppData\Local\Temp\ipykernel_12136\223693154.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  idle_time = pd.read_csv("Phil_with_idle_time_bike.csv")


In [6]:
#Times to datetime
idle_time["start_time"] = pd.to_datetime(idle_time["start_time"])
idle_time["end_time"] = pd.to_datetime(idle_time["end_time"])

#start_day feature
idle_time["start_day"] = idle_time["start_time"].dt.date


group = idle_time.groupby(["start_station", "start_day"])

#Count the trips
station_day_trip_counts = group.size().reset_index(name="trip_count")

#Average out over days
station_avg_trip_counts = station_day_trip_counts.groupby("start_station")["trip_count"].mean().reset_index(name="avg_trip_count")

idle_time_ext = pd.merge(idle_time, station_avg_trip_counts, on="start_station", how="left")

In [15]:
idle_time_ext.head()

,trip_id,duration,start_time,end_time,start_station,end_station,bike_id,plan_duration,trip_route_category,passholder_type,...,Hour_s,Date_e,Weekday_e,Hour_e,start_coordinates,end_coordinates,idle_time,start_day,avg_trip_count,idle_time_minutes
0,365315729,5,2021-01-03 15:03:00,2021-01-03 15:08:00,3211,3054,102,30,One Way,Indego30,...,15,2021-01-03,6,15,"39.971722,-75.170601","39.962502,-75.174202",0 days 07:23:00,2021-01-03,7.247222,7.383333
1,365351230,4,2021-01-03 22:31:00,2021-01-03 22:35:00,3054,3014,102,365,One Way,Indego365,...,22,2021-01-03,6,22,"39.962502,-75.174202","39.958858,-75.173691",0 days 19:25:00,2021-01-03,32.997249,19.416667
2,365375491,7,2021-01-04 18:00:00,2021-01-04 18:07:00,3014,3163,102,30,One Way,Indego30,...,18,2021-01-04,0,18,"39.958858,-75.173691","39.949741,-75.180969",25 days 21:11:00,2021-01-04,7.330914,621.183333
3,366822049,27,2021-01-30 15:18:00,2021-01-30 15:45:00,3004,3057,102,1,One Way,Day Pass,...,15,2021-01-30,5,15,"39.953781,-75.163742","39.96439,-75.179871",5 days 17:37:00,2021-01-30,11.382979,137.616667
4,367159233,17,2021-02-05 09:22:00,2021-02-05 09:39:00,3057,3111,102,365,One Way,Indego365,...,9,2021-02-05,4,9,"39.96439,-75.179871","39.977791,-75.213226",20 days 06:17:00,2021-02-05,28.447059,486.283333


In [7]:
idle_time_ext['avg_trip_count'].describe()

count    1.678842e+06
mean     2.015006e+01
std      9.605323e+00
min      1.577778e+00
25%      1.203558e+01
50%      1.930082e+01
75%      2.756749e+01
max      4.065295e+01
Name: avg_trip_count, dtype: float64

In [8]:
len(idle_time_ext)

1678842

In [9]:
#Check for NAs in idle_time
na_count = pd.isna(idle_time_ext["idle_time"]).sum()
print(na_count)
#10 NAs found, now convert NAs to zeros 
idle_time_ext["idle_time"].fillna(pd.Timedelta(seconds=0), inplace=True)

10


In [14]:
#Converting the str idle_time feature to minutes
idle_time_ext["idle_time"] = pd.to_timedelta(idle_time_ext["idle_time"])
#idle_time_ext["idle_time_minutes"] = idle_time_ext["idle_time"].dt.total_seconds() / 60
idle_time_ext["idle_time_minutes"] = idle_time_ext["idle_time"].dt.total_seconds().astype(int) /3600

In [16]:
#Adding 2 new features
idle_time_ext["idle_start_time"] = idle_time_ext["end_time"]
idle_time_ext["idle_end_time"] = idle_time_ext["end_time"] + idle_time_ext["idle_time"]

In [17]:
#Backup copy
idle_copy = idle_time_ext.copy()
len(idle_copy)

1678842

In [15]:
len(idle_time_ext[idle_time_ext["bike_id"] == 3289])

686

# New feature idle_bikes_counts
Idea is to add a new feature with counts the number of bikes for each trip that are also idle at the same station
AND at the same time called "idle_bikes_counts"

In [18]:
# New idle bikes for ALL bikes (too much computational power need): 
def count_idle_bikes_all(row):
    bike_idle_start_time = row["end_time"]
    bike_idle_end_time = bike_idle_start_time + row["idle_time"]

    # Filter the DataFrame to find bikes idle at the same end station during the idle time of the current bike
    bikes_idle_at_station = idle_copy[
        (idle_copy["bike_id"] != row["bike_id"]) &
        (idle_copy["end_station"] == row["end_station"]) &
        
        (((idle_copy["idle_start_time"] >= bike_idle_start_time) &
        (idle_copy["idle_start_time"] <= bike_idle_end_time)) |
         
        ((idle_copy["idle_end_time"] >= bike_idle_start_time) &
        (idle_copy["idle_end_time"] <= bike_idle_end_time)) |
         
        ((idle_copy["idle_start_time"] >= bike_idle_start_time) &
        (idle_copy["idle_end_time"] <= bike_idle_end_time))
        )
    ]
    return len(bikes_idle_at_station)

In [16]:
#calc idle bikes for only bike 3289 (works perfectly fine) and first correlation results showed his statistical relevance: 
def count_idle_bikes_3289(row):
    if row["bike_id"] == 3289:
        bike_idle_start_time = row["end_time"]
        bike_idle_end_time = bike_idle_start_time + row["idle_time"]

        # Filter the DataFrame to find bikes idle at the same end station during the idle time of bike 3289
        bikes_idle_at_station = idle_copy[
            (idle_copy["bike_id"] != 3289) &
            (idle_copy["end_station"] == row["end_station"]) &
            
            (((idle_copy["idle_start_time"] >= bike_idle_start_time) &
            (idle_copy["idle_start_time"] <= bike_idle_end_time)) |
             
            ((idle_copy["idle_end_time"] >= bike_idle_start_time) &
            (idle_copy["idle_end_time"] <= bike_idle_end_time)) |
             
            ((idle_copy["idle_start_time"] >= bike_idle_start_time) &
            (idle_copy["idle_end_time"] <= bike_idle_end_time))
            )
        ]
        return len(bikes_idle_at_station)
    else:
        return 0

In [19]:
# Apply functions from above to our dataset
idle_copy["idle_bikes_count"] = idle_copy.apply(count_idle_bikes_all, axis=1)
idle_copy.head()

KeyboardInterrupt: 

In [18]:
# Test out the new feature idle_bikes_count for 1 bike
np.set_printoptions(threshold=np.inf)

results = idle_copy[idle_copy["bike_id"] == 3289]["idle_bikes_count"].to_list()

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

test_results = pd.DataFrame(results)

test_results

,0
0,11
1,3
2,12
3,1
4,51
5,30
6,13
7,14
8,11
9,20


In [67]:
#len(idle_copy["idle_bikes_count"].isna())
null_count1 = idle_copy['idle_bikes_count'].isnull().sum()
zero_count = (idle_copy['idle_bikes_count'] != 0).sum().sum()

print(zero_count)
#print(null_count2)
len(idle_copy)

632


1678842